# Exploratory Data Analysis

Hypothesis: We can use the https://arxiv.org/pdf/2406.04313 method for increasing honesty

In [1]:
# autoreload your package
%load_ext autoreload
%autoreload 2
import adapter_overseer


In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [3]:
import warnings
# warnings.simplefilter("ignore")
# warnings.filterwarnings("ignore", ".*does not have many workers.*")
# warnings.filterwarnings("ignore", ".*divide by zero.*")

## numeric, plotting
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (7.0, 4)

## utils
from pathlib import Path
from tqdm.auto import tqdm
import logging, os, re
import collections, functools, itertools
from loguru import logger

from typing import List, Callable, Tuple, Dict, Optional
from jaxtyping import Float, Int
from torch import Tensor

# torch
# import pytorch_lightning as pl
from einops import rearrange, repeat, reduce
import torch
import torch.nn as nn


from baukit.nethook import get_module
from baukit import TraceDict

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

In [4]:
from adapter_overseer.config import ExtractConfig

cfg = ExtractConfig()
cfg

ExtractConfig(datasets=('amazon_polarity',), datasets_ood='imdb', model='failspy/Llama-3-8B-Instruct-abliterated', collection_layers=('base_model.model.model.layers.10', 'base_model.model.model.layers.20'), batch_size=2, prompt_format=None, num_shots=2, max_length=776, max_examples=1000, seed=42, max_epochs=1)

## Load

In [5]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# https://huggingface.co/blog/mlabonne/orpo-llama-3
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16
torch_dtype, device

(torch.bfloat16, device(type='cuda', index=0))

In [7]:
# load model
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
quantization_config = BitsAndBytesConfig(load_in_4bit=True,     bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch_dtype, bnb_4bit_use_double_quant=True,)
model = AutoModelForCausalLM.from_pretrained(cfg.model, device_map="auto", quantization_config=quantization_config,)
model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(cfg.model)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = 'left'
tokenizer.truncation_side = 'left'

In [ ]:
# https://old.reddit.com/r/LocalLLaMA/comments/1coizjy/tokenizer_config_of_llama3_changed_by_meta_in_hf/
tokenizer.eos_token # it's good

In [ ]:
# from peft import get_peft_config, get_peft_model, LoraConfig, TaskType
# \peft_config = LoraConfig(
#     task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1
# )
# model = get_peft_model(model, peft_config)


In [ ]:
# from peft import prepare_model_for_int8_training
# # we need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in float32 for stability. We also cast the output of the last layer in float32 for the same reasons.
# model = prepare_model_for_int8_training(model, output_embedding_layer_name="proj_out")

In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model
# https://github.com/huggingface/peft/blob/main/src/peft/utils/constants.py
config = LoraConfig(
                        #r=32,
                         lora_alpha=10,  # from paper
                    # target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none"
                    )

from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

model = get_peft_model(model, config)
model.print_trainable_parameters()

# check adapter disabling works
with model.disable_adapter():
    model.print_trainable_parameters()

## Get data

In [ ]:
# perhaps use load_preproc_datasets from sdb_probes_are_lie_detectors repo... /media/wassname/SGIronWolf/projects5/elk/sgd_probes_are_lie_detectors/src/prompts/prompt_loading.py

In [ ]:
# load a dataset of paired prompts, to try and get the model to lie
from adapter_overseer.prompts.prompt_loading import load_preproc_datasets

N = cfg.max_examples
ds_tokens = load_preproc_datasets(
    cfg.datasets,
    tokenizer,
    N=N,
    seed=cfg.seed,
    num_shots=cfg.num_shots,
    max_length=cfg.max_length,
    prompt_format=cfg.prompt_format,
)
ds_tokens


## Train: transformers

https://github.com/huggingface/peft/blob/main/examples/int8_training/Finetune_opt_bnb_peft.ipynb

In [ ]:
# TODO change the loss function!
# we need to modify the forward pass, so that it returns a different loss function
# but to calculate this we will need to residuals now, and as they werre
# loss_bad = mse(repr_current, repr_target)

# from transformers import SFTTrainer
from trl.trainer import SFTTrainer, SFTConfig
import torch.nn.functional as F

from adapter_overseer.helpers.torch_helpers import clear_mem, switch
from adapter_overseer.helpers.scores import select_choices

class CustomSFTTrainer(SFTTrainer):
    """
    Custom SFTTrainer that orthoganalizes the repr of bad examples, and retains good repr of examples

    See: https://arxiv.org/pdf/2406.04313

    args:
        collection_layers: list of baukit layer names to collect
    """
    def __init__(self, *args, collection_layers: list, alpha=0.1, **kwargs):
        super(CustomSFTTrainer, self).__init__(*args, **kwargs)
        self.collection_layers = collection_layers
        self.alpha = alpha
        self.total_steps = self.args.max_steps

    def compute_loss(self, model, inputs, return_outputs=False):       

        batch = {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']}

        # collect the residuals of the model
        with model.disable_adapter():
            orig_outputs = model(**batch, output_hidden_states=True)
        outputs = model(**batch, output_hidden_states=True)

        def collect_hs(hidden_states):
            """The residual stream is the diff of the hs."""
            hs = [hidden_states[i] for i in self.collection_layers]
            return rearrange(hs, 'l b t h -> b l t h').diff(1)

        rep_adapt = collect_hs(outputs.hidden_states)
        rep_orig = collect_hs(orig_outputs.hidden_states)

        # so now we have a mixed batch of good and bad outputs
        # get probs of each choice
        # compare to labels to seperate into good and bad
        choice_ids = inputs['choice_ids'].detach().cpu().long()
        # label_instructed = inputs['label_true'] ^ inputs['instructed_to_lie']
        label_true = inputs['label_true']

        # does the underlying model get it right or wrong?
        end_logits = orig_outputs["logits"][:, -1]
        probs = torch.softmax(end_logits, -1)
        choice_probs = select_choices(probs, choice_ids).sum(2)
        binary_ans = choice_probs[:, 1] / (choice_probs.sum(1) + 1e-12)
        correct_truth_telling = switch(binary_ans, label_true)
        # correct_instruction_following = switch(binary_ans, label_instructed)

        mask_desired = correct_truth_telling>0.5


        # get coeffecient
        steps = self.state.global_step + 1
        c = torch.tensor(self.alpha * steps / (2 * self.total_steps)).to(rep_orig.dtype)
        loss_retain = F.mse_loss(rep_orig, rep_adapt, reduction='none' )[mask_desired]
        if loss_retain.numel() == 0:
            loss_retain = 0
        else:
            loss_retain = loss_retain.mean()
        loss_rr = F.relu(F.cosine_similarity(rep_orig, rep_adapt, dim=1))[~mask_desired]
        if loss_rr.numel() == 0:
            loss_rr = 0
        else:
            loss_rr = loss_rr.mean()
        loss = loss_rr * (1 - c) + c * loss_retain
        loss = loss
        logger.debug(f"steps: {steps}, c: {c}, loss_rr: {loss_rr:2.3f}, loss_retain: {loss_retain:2.3f}, loss={loss:2.3f}, mask_desired: {(mask_desired*1.0).mean():2.3f}")
        
        return (loss, outputs) if return_outputs else loss
    

# TODO make sure that multiple cols get passed into trainer
ds = ds_tokens.select_columns(['label_true', 'label_instructed' ,'instructed_to_lie', 'input_ids', 'attention_mask', 'choice_ids'])

import transformers

# see https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py#L58
trainer = CustomSFTTrainer(
    model=model,
    train_dataset=ds,
    collection_layers=[10, 20],
    # max_seq_length=cfg.max_length,
    args=SFTConfig(
        # see https://github.com/huggingface/trl/blob/main/trl/trainer/sft_config.py#L21
        max_seq_length=cfg.max_length,
        per_device_train_batch_size=4, # 18GB/24GB
        gradient_accumulation_steps=4, # we want to accumulate the gradients to make the batch size larger, so we have sufficient examples of good and bad behaviour to learn from
        warmup_steps=10,
        max_steps=150*4, # 150 steps of batch=16 in paper
        learning_rate=1e-3, # from paper
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        remove_unused_columns=False,
    ),
    # data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

In [ ]:
# save
model.save_pretrained("../outputs/hs_adapter")

# Eval

In [ ]:
from datasets import load_dataset
# multiple_choice
from torch.utils.data import DataLoader
# dataset = load_dataset("truthfulqa/truthful_qa", "multiple_choice")

# HACK it was stalling for hours, so I loaded it locally
dataset = load_dataset("../data/truthful_qa")['validation']
dataset

In [ ]:

# print(row)

def format_prompt(row):
    prompt = f"Q: {row['question']}\n"
    for i, choice in enumerate(row['mc1_targets']['choices']):
        prompt += f"{i+1}. {choice}\n"

    choices = [str(i) for i in range(len(row['mc1_targets']['labels']))]
    return {'text': prompt, 
            'label': [np.argmax(row['mc1_targets']['labels'])],
            'choices': choices,
            'num_choices': len(choices),
            }

dataset1 = dataset.map(format_prompt)

In [ ]:
max([len(r['labels']) for r in dataset['mc1_targets']])

In [ ]:
# get our choice ids
choices = [str(i) for i in range(13)]
choice_ids = [tokenizer(c, add_special_tokens=False).input_ids[0] for c in choices]
choice_ids

In [ ]:

def tokenization(example):
    o = tokenizer(example["text"], padding="max_length", truncation=True, max_length=cfg.max_length, return_tensors="pt")
    return o

dataset2 = dataset1.map(tokenization, batched=True).select_columns([ 'label', 'input_ids', 'attention_mask', 
                                                                    'num_choices'
                                                                    ]).with_format("torch")
dataset2

In [ ]:
# https://github.dev/sylinrl/TruthfulQA/blob/fdd8ad1c0d00a478cf8b0bb41a3ad8378c16293b/truthfulqa/models.py#L311


probs = []
base_probs = []

dl = DataLoader(
    dataset2, batch_size=4, num_workers=0)
for b in tqdm(dl):
    inputs = {'input_ids': b['input_ids'], 'attention_mask': b['attention_mask']}
    with torch.no_grad():
        with model.disable_adapter():
            out_base = model(**inputs)
        out = model(**inputs)

        for j in range(len(out["logits"])):
            n = b['num_choices'][j]
            b_choice_ids = choice_ids[:n]
            label = b['label'][j, 0]

            choice_probs_base = out_base["logits"][j, -1, b_choice_ids].softmax(dim=-1)
            choice_probs = out["logits"][j, -1, b_choice_ids].softmax(dim=-1)
            prob = choice_probs[label].item()
            prob_base = choice_probs_base[label].item()
            probs.append(prob)
            base_probs.append(prob_base)
        

In [ ]:
choice_probs

In [ ]:
choice_probs_base

In [ ]:
acc = ((torch.tensor(probs)>0.5)*1.0).mean()
base_acc = ((torch.tensor(base_probs)>0.5)*1.0).mean()
acc, base_acc

In [ ]:
prob_correct = torch.tensor(probs).mean()
prob_base_correct = torch.tensor(base_probs).mean()
prob_correct, prob_base_correct

In [ ]:
b

In [ ]:
from datasets import load_dataset
# import evaluate
from transformers import pipeline

# accuracy = evaluate.load("accuracy")

dataset_eval = load_dataset("EleutherAI/truthful_qa_binary", split="test")["text"]

In [ ]:
from evaluate import evaluator
from datasets import load_dataset
data = load_dataset("EleutherAI/truthful_qa_binary", split="validation[:20]")
task_evaluator = evaluator("text-classification")
# see https://huggingface.co/docs/evaluate/v0.4.0/en/package_reference/evaluator_classes#evaluate.TextClassificationEvaluator

pipe = pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
    # feature_extractor=feature_extractor,
    # device=device,
)


results = task_evaluator.compute(
    pipe,
    # tokenizer=tokenizer,
    data=data,
    metric="accuracy",
    input_column="question",
    # label_mapping={"LABEL_0": 0.0, "LABEL_1": 1.0},
    random_state=42
)

In [ ]:

# def preprocess_function(examples):
#     return tokenizer(examples["text"], truncation=True)

# https://huggingface.co/docs/transformers/v4.41.3/en/main_classes/pipelines#pipeline-batching
pipe = pipeline("text-classification", device="auto", model=model, tokenizer=tokenizer)
batch_size = 4
for out in tqdm(pipe(dataset_eval, batch_size=batch_size), total=len(dataset_eval)):
    1/0
    pass

In [ ]:
model.train()
model.print_trainable_parameters()

In [ ]:
with model.disable_adapter():
    model.print_trainable_parameters()